# 웹페이지 크롤링 이후 RAG로 활용해보기

In [ ]:
!pip install chromadb tiktoken transformers sentence_transformers openai langchain pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 55.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 45.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 63.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 82.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 3.8 MB/s eta 0:00:0

In [ ]:
!pip install -U langchain-community unstructured

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 15.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 50.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 53.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.7/409.7 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.9/586.9 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.9/274.9 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 55.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.9/62.9 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.9/434.9 kB 24.7 MB/s eta 0:00:00
   ━━

In [ ]:
import os
import openai
os.environ["OPENAI_API_KEY"] = 'your API KEY'

In [ ]:
import tiktoken

tokenizer = tiktoken.get_encoding("cl100k_base")

def tiktoken_len(text):
    tokens = tokenizer.encode(text)
    return len(tokens)

In [ ]:
from bs4 import BeautifulSoup
import requests

# 크롤링할 URL 목록
urls = [
    "https://creation.kr/EvidenceofFlood/?idx=1288454&bmode=view",
    "http://creation.kr/EvidenceofFlood/?idx=1288394&bmode=view",
    "http://creation.kr/EvidenceofFlood/?idx=1288412&bmode=view",
    "http://creation.kr/Geology/?idx=1290484&bmode=view",
    "http://creation.kr/EvidenceofFlood/?idx=1757332&bmode=view",
    "http://creation.kr/EvidenceofFlood/?idx=1288377&bmode=view",
    "http://creation.kr/EvidenceofFlood/?idx=1288391&bmode=view",
    "http://creation.kr/BiblicalChronology/?idx=1289277&bmode=view",
    "http://creation.kr/EvidenceofFlood/?idx=1288384&bmode=view",
    "http://creation.kr/EvidenceofFlood/?idx=1288365&bmode=view",
    "http://creation.kr/EvidenceofFlood/?idx=1288378&bmode=view",
    "http://creation.kr/EvidenceofFlood/?idx=1288390&bmode=view",
    "http://creation.kr/EvidenceofFlood/?idx=1288412&bmode=view",
    "http://creation.kr/EvidenceofFlood/?idx=1288394&bmode=view",
    "http://creation.kr/EvidenceofFlood/?idx=1288411&bmode=view",
    "http://creation.kr/EvidenceofFlood/?idx=1288373&bmode=view",
    "http://creation.kr/EvidenceofFlood/?idx=1288427&bmode=view",
    "http://creation.kr/EvidenceofFlood/?idx=1288375&bmode=view",
    "http://creation.kr/EvidenceofFlood/?idx=1288370&bmode=view",
    "http://creation.kr/EvidenceofFlood/?idx=1288415&bmode=view",
    "http://creation.kr/QnA/?idx=1828050&bmode=view",
    "http://creation.kr/Catastrophic/?idx=4527739&bmode=view",
    "http://creation.kr/QnA/?idx=1828065&bmode=view",
    "http://creation.kr/Ark/?idx=1288301&bmode=view",
    "http://creation.kr/EvidenceofFlood/?idx=1288384&bmode=view",
    "http://creation.kr/EvidenceofFlood/?idx=1288433&bmode=view",
    "http://creation.kr/EvidenceofFlood/?idx=1288444&bmode=view",
    "http://creation.kr/Catastrophic/?idx=1288267&bmode=view",
    "http://creation.kr/EvidenceofFlood/?idx=1288460&bmode=view",
    "http://creation.kr/EvidenceofFlood/?idx=1288395&bmode=view",
    "http://creation.kr/EvidenceofFlood/?idx=1288477&bmode=view",
    "http://creation.kr/EvidenceofFlood/?idx=1288419&bmode=view",
    "http://creation.kr/QnA/?idx=1828017&bmode=view",
    "http://creation.kr/EvidenceofFlood/?idx=1288472&bmode=view",
    "http://creation.kr/Sediments/?idx=1288667&bmode=view",
    "http://creation.kr/EvidenceofFlood/?idx=1288481&bmode=view",
    "http://creation.kr/EvidenceofFlood/?idx=2094916&bmode=view",
    "http://creation.kr/EvidenceofFlood/?idx=1288476&bmode=view",
    "http://creation.kr/Sediments/?idx=1288666&bmode=view",
    "http://creation.kr/EvidenceofFlood/?idx=1288473&bmode=view",
    "http://creation.kr/Sediments/?idx=1288689&bmode=view",
    "http://creation.kr/EvidenceofFlood/?idx=3870071&bmode=view",
    "http://creation.kr/EvidenceofFlood/?idx=1288470&bmode=view",
    "http://creation.kr/EvidenceofFlood/?idx=2094916&bmode=view",
    "http://creation.kr/Sediments/?idx=1288676&bmode=view",
    "http://creation.kr/EvidenceofFlood/?idx=1288474&bmode=view",
    "http://creation.kr/Sediments/?idx=1288628&bmode=view",
    "http://creation.kr/Sediments/?idx=1288599&bmode=view",
    "http://creation.kr/Sediments/?idx=1288685&bmode=view",
    "http://creation.kr/Sediments/?idx=1288682&bmode=view",
    "http://creation.kr/EvidenceofFlood/?idx=1288475&bmode=view",
    "http://creation.kr/EvidenceofFlood/?idx=1288486&bmode=view",
    "http://creation.kr/Sediments/?idx=1288671&bmode=view",
    "http://creation.kr/Sediments/?idx=1288680&bmode=view",
    "http://creation.kr/Sediments/?idx=1288681&bmode=view",
    "http://creation.kr/Sediments/?idx=1288677&bmode=view",
    "http://creation.kr/EvidenceofFlood/?idx=1288487&bmode=view",
    "http://creation.kr/EvidenceofFlood/?idx=1288443&bmode=view",
    "http://creation.kr/EvidenceofFlood/?idx=1288478&bmode=view",
    "http://creation.kr/EvidenceofFlood/?idx=1288397&bmode=view",
    "http://creation.kr/QnA/?idx=1828072&bmode=view",
    "http://creation.kr/EvidenceofFlood/?idx=1288431&bmode=view",
    "http://creation.kr/Sediments/?idx=1288679&bmode=view",
    "http://creation.kr/EvidenceofFlood/?idx=1288454&bmode=view",
    "http://creation.kr/EvidenceofFlood/?idx=1288392&bmode=view",
    "http://creation.kr/EvidenceofFlood/?idx=1288461&bmode=view",
    "http://creation.kr/Sediments/?idx=1288662&bmode=view",
    "http://creation.kr/EvidenceofFlood/?idx=1288451&bmode=view",
    "http://creation.kr/EvidenceofFlood/?idx=1288452&bmode=view",
    "http://creation.kr/Sediments/?idx=1288601&bmode=view",
    "http://creation.kr/Ark/?idx=1288309&bmode=view ",
    "http://creation.kr/EvidenceofFlood/?idx=2511178&bmode=view",
    "http://creation.kr/EvidenceofFlood/?idx=1288445&bmode=view",
    "http://creation.kr/EvidenceofFlood/?idx=1288372&bmode=view",
    "http://creation.kr/EvidenceofFlood/?idx=1288387&bmode=view",
    "http://creation.kr/Genesis/?idx=2851501&bmode=view",
    "http://creation.kr/EvidenceofFlood/?idx=1288407&bmode=view",
    "http://creation.kr/EvidenceofFlood/?idx=1288389&bmode=view",
    "http://creation.kr/EvidenceofFlood/?idx=1288417&bmode=view",
    "http://creation.kr/EvidenceofFlood/?idx=1288383&bmode=view",
    "http://creation.kr/Variation/?idx=1290378&bmode=view",
    "http://creation.kr/Variation/?idx=1290401&bmode=view",
    "http://creation.kr/BiblenHistory/?idx=1288908&bmode=view",
    "http://creation.kr/Genesis/?idx=1289137&bmode=view",
    "http://creation.kr/BiblenHistory/?idx=4082749&bmode=view",
    "http://creation.kr/EvidenceofFlood/?idx=1288381&bmode=view",
    "http://creation.kr/BiblenHistory/?idx=2881356&bmode=view",
    "http://creation.kr/Mutation/?idx=1289856&bmode=view",
    "http://creation.kr/IntelligentDesign/?idx=1291576&bmode=view",
    "http://creation.kr/Genesis/?idx=1289144&bmode=view",
    "http://creation.kr/BiblenHistory/?idx=1289005&bmode=view",
    "http://creation.kr/BiblenHistory/?idx=1288993&bmode=view",
    "http://creation.kr/Genesis/?idx=1289103&bmode=view",
    "http://creation.kr/Mutation/?idx=1289832&bmode=view",
    "http://creation.kr/IntelligentDesign/?idx=1291722&bmode=view",
    "http://creation.kr/Mutation/?idx=1757411&bmode=view"
]

# User-Agent 헤더 설정
headers = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
        "(KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36"
    )
}

# 추출한 텍스트를 저장할 파일 열기
with open('output.txt', 'w', encoding='utf-8') as file:
    # 각 URL에 대해 텍스트 추출
    for url in urls:
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            paragraphs = soup.find_all('p')  # 모든 <p> 태그에서 텍스트 추출
            content = "\n".join([para.get_text(strip=True) for para in paragraphs])
            # URL과 해당 내용 파일에 기록
            file.write(f"URL: {url}\n{content}\n\n")
        else:
            print(f"Failed to fetch the webpage: {url}. Status code: {response.status_code}")
